In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[6]:


import datetime as dt
from datetime import date,timedelta
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from sqlalchemy import create_engine
import urllib


# In[7]:


#Connect to Staging database
server   = 'tcp:cliq.database.windows.net'
database = 'CLIQ_Staging_DB'
username = 'cliq_BI_Dev01'
password = 'ShahranGirl_2020'

params = urllib.parse.quote_plus(

 'DRIVER={ODBC Driver 17 for SQL Server};'+ 

 'SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

CLIQengine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)


# In[8]:


#Connect to Metrics database
server   = 'tcp:cliq.database.windows.net'
database = 'CLIQ_Metrics'
username = 'cliq_admin'
password = 'gfd#2019'

params = urllib.parse.quote_plus(

 'DRIVER={ODBC Driver 17 for SQL Server};'+ 

 'SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

Metricsengine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)


# In[9]:


# LPA data extract
try:
    with CLIQengine.begin() as conn:
        conn.execute("exec metricDB.LPADataExtractForMetricDB")
except:
    pass

L = """SELECT * FROM ##LPA"""
LPA = pd.read_sql_query(L,CLIQengine)
LPA.to_sql(name ='LPA',schema='stg', con=Metricsengine, if_exists='replace', index=False)


# In[11]:



#STORED PROC IN METRICS DB TO UPDATE DATA, CONSTRAINTS AND PK FK RELATIONSHIP

try:
    with Metricsengine.begin() as conn:
        conn.execute("exec Report.LayeredProcessAudit")
except:
    pass

